In [32]:
import fitz  # pip3 install pymupdf
import time
import re
import os


def get_image(path, pic_path):
    '''從pdf中提取圖片
    :param path: pdf的路徑
    :param pic_path: 圖片保存的路徑
    :return: 無return
    '''
    t0 = time.clock()
    # 00、使用正則表達式查找PDF中的圖片
    checkXO = r"/Type(?= */XObject)"
    checkIM = r"/Subtype(?= */Image)"

    # 一、打開pdf，打印PDF的相關信息
    doc = fitz.open(path)
    # 圖片計數
    imgcount = 0
    lenXREF = doc._getXrefLength()

    # 打印PDF的信息
    print("文件名:{}, 頁數: {}, 對象: {}".format(path, len(doc), lenXREF - 1))

    # 二、遍歷PDF中的對象，遇到是圖像才進行下一步，不然就continue
    for i in range(1, lenXREF):
        # 定義對象字符串
        # text = doc._getObjectString(i)
        text = doc._getXrefString(i)
        # print(text)
        # continue
        isXObject = re.search(checkXO, text)
        # 使用正則表達式查看是否是圖片
        isImage = re.search(checkIM, text)
        # 如果不是對象也不是圖片，則continue
        if not isXObject or not isImage:
            print("不是圖片")
            continue
        imgcount += 1
        # 根據索引生成圖像
        pix = fitz.Pixmap(doc, i)
        # 根據pdf的路徑生成圖片的名稱
        new_name = path.replace('\\', '_') + "_img{}.png".format(imgcount)
        new_name = new_name.replace(':', '')

        # 三、將圖像存爲png格式
        # 如果pix.n<5,可以直接存爲PNG
        if pix.n < 5:
            pix.writePNG(os.path.join(pic_path, new_name))
        # 否則先轉換CMYK
        else:
            pix0 = fitz.Pixmap(fitz.csRGB, pix)
            pix0.writePNG(os.path.join(pic_path, new_name))
            pix0 = None
        # 釋放資源
        pix = None
    t1 = time.clock()
    print("運行時間:{}s".format(t1 - t0))
    print("提取了{}張圖片".format(imgcount))


# 運行
if __name__=='__main__':
    # pdf路徑
    path = r"C:\Users\ctkri\Desktop\DAFRAM\DAFRAM spa_2020 rev 00.pdf"  # 更改路徑可使用
    pic_path = r"C:\Users\ctkri\Desktop\DAFRAM\DAFRAM"
    # 創建保存圖片的文件夾
    if os.path.exists(pic_path):
        print("文件夾已存在，請重新創建新文件夾！")
        raise SystemExit
    else:
        os.mkdir(pic_path)
    get_image(path, pic_path)

文件夾已存在，請重新創建新文件夾！


SystemExit: 